Trip log joins are an operational necessity. However, they do not process fast enough. Let's get them running more quickly.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.transit import gtfs_realtime_pb2
import sys; sys.path.append("../src/")
from processing import parse_feeds_into_trip_logbook, merge_trip_logbooks

with open("../src/tests/data/gtfs_realtime_pull_1.dat", "rb") as f:
    gtfs_r0 = gtfs_realtime_pb2.FeedMessage()
    gtfs_r0.ParseFromString(f.read())
with open("../src/tests/data/gtfs_realtime_pull_2.dat", "rb") as f:
    gtfs_r1 = gtfs_realtime_pb2.FeedMessage()
    gtfs_r1.ParseFromString(f.read())

left_logbook = parse_feeds_into_trip_logbook([gtfs_r0], [0])
right_logbook = parse_feeds_into_trip_logbook([gtfs_r1], [1])

In [3]:
# Slow!
# result = merge_trip_logbooks([left_logbook, right_logbook])

The above makes it obvious that the problem is that we are running `_join_trip_logs`, a multi-second operation, 162 times here. So we need to raise that functions' execution speed.

In [4]:
from processing import _join_trip_logs

Old code:

This is a 20x speedup. However, the full merge still takes 20 seconds (!), due to `__setitem__` calls still in `_join_trip_logs`. It's not transparent to me where these are coming from, so I spent some time muddling about in it.

We got another 33% speedup by avoiding `DataFrame` transforms.

Eventually we get a really huge speedup, from forever to less than a second, but this is still surprising to me as I expected it to process faster. But this seems to be as fast as it gets...

In [6]:
from pyinstrument import Profiler

profiler = Profiler()
profiler.start()

# code you want to profile
result = merge_trip_logbooks([left_logbook, right_logbook])

profiler.stop()

print(profiler.output_text(unicode=True, color=True))

0.862 _join_trip_logs  processing.py:592
├─ 0.336 __setitem__  pandas/core/indexing.py:135
│  ├─ 0.186 _setitem_with_indexer  pandas/core/indexing.py:233
│  │  ├─ 0.115 setter  pandas/core/indexing.py:455
│  │  │  ├─ 0.059 __setitem__  pandas/core/frame.py:2405
│  │  │  │  └─ 0.049 _set_item  pandas/core/frame.py:2473
│  │  │  │     ├─ 0.030 _set_item  pandas/core/generic.py:1499
│  │  │  │     │  └─ 0.028 set  pandas/core/internals.py:3636
│  │  │  │     └─ 0.018 _sanitize_column  pandas/core/frame.py:2593
│  │  │  ├─ 0.026 setitem  pandas/core/internals.py:3167
│  │  │  │  └─ 0.026 apply  pandas/core/internals.py:2978
│  │  │  │     └─ 0.022 setitem  pandas/core/internals.py:654
│  │  │  └─ 0.018 copy  pandas/core/generic.py:3057
│  │  │     └─ 0.013 copy  pandas/core/internals.py:3364
│  │  ├─ 0.036 __getitem__  pandas/core/frame.py:2035
│  │  │  └─ 0.032 _getitem_column  pandas/core/frame.py:2061
│  │  │     └─ 0.032 _get_item_cache  pandas/core/generic.py:1381
│  │  │        ├─ 0.

In [7]:
import os

logs = [f for f in os.listdir("./data/subway_time_20160512") if f != 'arch.tar.xz' 
        and 'si' not in f and 'l' not in f]

In [8]:
logs[:5]

['gtfs-20160512T0415Z',
 'gtfs-20160512T1759Z',
 'gtfs-20160512T2155Z',
 'gtfs-20160512T0610Z',
 'gtfs-20160513T0153Z']

In [9]:
from google.transit import gtfs_realtime_pb2

In [10]:
def parse_feed(filepath):
    with open(filepath, "rb") as f:
        try:
            fm = gtfs_realtime_pb2.FeedMessage()
            fm.ParseFromString(f.read())
            return fm
        except (KeyboardInterrupt, SystemExit):
            raise
        except:
            return None

Note: the data above comes from the next notebook. You can get it yourself by using the following magical incantation:

    pip install requests gtfs-realtime-bindings
    python -c "import requests; r = requests.get('http://data.mytransit.nyc.s3.amazonaws.com/subway_time/2016/2016-05/subway_time_20160512.tar.xz'); open('arch.tar.xz', 'wb').write(r.content)"
    tar xvfJ arch.tar.xz
    python -c "from google.transit import gtfs_realtime_pb2; test_example = gtfs_realtime_pb2.FeedMessage().ParseFromString(open('gtfs-20160512T0400Z', 'rb').read()); print(type(test_example))"

In [11]:
from tqdm import tqdm

In [12]:
feeds = [parse_feed("./data/subway_time_20160512/" + l) for l in tqdm(logs[:6])]

100%|██████████| 6/6 [00:01<00:00,  4.33it/s]


In [13]:
import sys; sys.path.append("../src/")
from processing import parse_feeds_into_trip_logbook

In [16]:
logbooks = [
    parse_feeds_into_trip_logbook(feeds[0:3], [0, 1, 2]), 
    parse_feeds_into_trip_logbook(feeds[3:6], [3, 4, 5])
]

This is now very fast.

In [19]:
%timeit master = merge_trip_logbooks(logbooks)

1 loop, best of 3: 305 ms per loop
